In [ ]:
# 첫 번째 셀
import sys
sys.path.append('../core')

from mcp_client import MCPClient
import asyncio
import json

# 두 번째 셀 - 포트 수정하여 연결 테스트
async def test_connection():
    # 서버가 8000 포트에서 실행 중이므로 포트 수정
    client = MCPClient(server_url="http://localhost:8000")
    try:
        success = await client.connect()
        if success:
            print("✅ 연결 성공!")
            info = await client.get_server_info()
            print("서버 정보:")
            print(json.dumps(info, ensure_ascii=False, indent=2))
        else:
            print("❌ 연결 실패")
    finally:
        await client.disconnect()

# 실행
await test_connection()

In [ ]:
# test_client.py에 추가하거나 새로운 함수로 테스트
async def test_tool_call():
    client = MCPClient(server_url="http://localhost:8000")
    try:
        await client.connect()
        print("🔧 도구 호출 테스트...")
        result = await client.call_tool("get_env_info")
        print("도구 호출 결과:")
        print(json.dumps(result, ensure_ascii=False, indent=2))
        return result
    finally:
        await client.disconnect()


await test_tool_call()

In [ ]:
# 첫 번째 셀
import sys
sys.path.append('../core')

from langchain_agent import LangChainAgent
from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

# 두 번째 셀 - Bitget 도구 테스트
async def test_bitget_tools():
    agent_client = LangChainAgent(model_key="gpt4")
    mcp_server_url = agent_client.mcp_server_url
    
    async with streamablehttp_client(mcp_server_url) as (read, write, _):
        async with ClientSession(read, write) as session:
            await session.initialize()
            
            # 에이전트 생성
            agent = await agent_client.create_agent(session, "bitget_agent")
            if not agent:
                print("❌ 에이전트 생성 실패")
                return
            
            print("✅ Bitget 도구 포함 에이전트 생성 완료")
            
            # Bitget 관련 질문들
            questions = [
                "현재 잔고를 확인해줘.",                
                "리플을 가용 밸런스 내에서 숏포지션을 최소로 구매해줘.",                
                "현재 미체결 주문 확인해줘.",                
                "미체결 주문이 있다면 취소해줘",                
                "현재 포지션을 확인해줘.",
                "모든 포지션을 청산해줘."
            ]
            
            for question in questions:
                print(f"\n👤 질문: {question}")
                answer = await agent_client.ask_agent(question, "bitget_agent")
                print(f"🤖 답변: {answer}")

# 실행
await test_bitget_tools()